# Simulation of Rayleigh-Taylor Instabilities Using Python

## Team Members
- Jack Bollenbacher
- Sahat Jaldu
- Riddhi Chhabra

**Note**: John Westfall is listed as a teammate but he had absolutely *no* part in developing this project. He did not return any communications, as was discussed with Dr. Xu

## Introduction
Rayleigh-Taylor instabilities (RTI) arise in fluid systems where a denser fluid overlays a less dense fluid. This configuration is inherently unstable, as the gravitational potential energy in the system can decrease if the denser fluid moves downward and the lighter fluid moves upward. If a perturbation is introduced at the interface between the fluids, the system responds by amplifying the disturbance, leading to the characteristic "fingers" or "spikes" of the denser fluid descending into the lighter fluid, accompanied by "bubbles" of the lighter fluid rising.

## RTI is a significant phenomenon because:

>In astrophysics, it helps explain mixing processes during supernova explosions and stellar evolution.

>In geophysics, it contributes to magma dynamics and mantle convection.

>In engineering, it provides a challenge in processes such as inertial confinement fusion, where controlling these instabilities is crucial for achieving stable energy confinement.

## Problem Description and Methods
The goal is to simulate and visualize the Rayleigh-Taylor instability by solving the governing equations of fluid dynamics on a discretized grid.

### Governing Equations
1. **Navier-Stokes Equation**:
   $
   \frac{\partial \mathbf{v}}{\partial t} + (\mathbf{v} \cdot \nabla)\mathbf{v} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{v} + \mathbf{g}
   $
2. **Continuity Equation**:
   $
   \nabla \cdot \mathbf{v} = 0
   $
3. **Density Advection**:
   $
   \frac{\partial \rho}{\partial t} + \mathbf{v} \cdot \nabla \rho = 0
   $
4. **Vorticity Transport**:
   $
   \frac{\partial \omega}{\partial t} + \mathbf{v} \cdot \nabla \omega = \nu \nabla^2 \omega
   $

### Numerical Approach
- **Grid Discretization**: The simulation space is represented as a uniform grid.
- **Initialization**: The density, velocity, and vorticity fields are initialized, with a localized impulse added to seed instabilities.
- **Streamfunction and Velocity**: The vorticity equation is solved iteratively to derive the streamfunction and velocity fields.
- **Time Integration**: Fields are updated using explicit time-stepping methods.
- **Visualization**: The simulation is visualized using Python's `matplotlib` and `FuncAnimation`.

## Code
The code includes detailed comments explaining each step of the simulation process. It initializes the density and velocity fields, solves the governing equations, and visualizes the results.

## Results
- **Animation**: The evolution of the fluid density field reveals the formation and growth of instabilities at the interface.

## Discussion
The simulation demonstrates the basic dynamics of Rayleigh-Taylor instabilities. The downward acceleration of the denser fluid into the lighter fluid creates characteristic patterns. However, numerical diffusion and resolution limitations suppress some finer-scale features. Future improvements could include adaptive meshing and more advanced numerical solvers. We could also standardize the simulation units to SI so that it could simulate real life events, like RT-instability in inertial confinement fusion

## References
1. Chandrasekhar, S. (1961). *Hydrodynamic and Hydromagnetic Stability*.
2. Python `numpy` documentation.
3. Python `matplotlib` documentation.
4. https://www.youtube.com/watch?v=E9_kyXjtRHc extremely helpful youtube video

## Contributions
- **Jack B**: Put together mathematical "pseudo-pseudocode", created and debugged final simulation version, put together this report
- **Riddhi C**: Worked on initial simulation and building off of the math foundations.
- **Sahat J**: Made second version simulation, researched different types of simulation methods (and ultimately found the one we used)



In [33]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Parameters
nx, ny = 100, 200  # Grid resolution
Lx, Ly = 1.0, 2.0  # Domain size
dx, dy = Lx / nx, Ly / ny
dt = 0.00001  # Time step
t_max = 0.25  # Total simulation time
nu = 0.001  # Viscosity
g = 9.8  # Gravitational acceleration
rho_light, rho_heavy = 1, 20  # Fluid densities

# Create grid/simulation space
x = np.linspace(0, Lx, nx)
y = np.linspace(0, Ly, ny)
X, Y = np.meshgrid(x, y)

# Initialize density field heavy on top, light in the bottom
rho = np.ones((ny, nx)) * rho_heavy
rho[Y < Ly / 2] = rho_light

# Introduce a localized velocity impulse to start the instability
u = np.zeros((ny, nx))  # Velocity in x-direction
v = np.zeros((ny, nx))  # Velocity in y-direction
impulse_region = (np.abs(X - Lx / 2) < 0.1) & (np.abs(Y - Ly / 2) < 0.1)  # Central impulse
v[impulse_region] = -500.0  # Strong downward velocity impulse

# Seed initial vorticity to add curl to the initial instability
omega = np.zeros((ny, nx))  # Vorticity
omega[impulse_region] = 1  # Moderate curl

# Initialize streamfunction for Navier Stokes
psi = np.zeros((ny, nx))  # Streamfunction

# Store initial boundary values, this was done to stop artifacts from forming at the edges
initial_top = rho[0, :].copy()
initial_bottom = rho[-1, :].copy()
initial_left = rho[:, 0].copy()
initial_right = rho[:, -1].copy()

# Helper functions
def laplacian(field):
    return (
        np.roll(field, 1, axis=0) + np.roll(field, -1, axis=0) +
        np.roll(field, 1, axis=1) + np.roll(field, -1, axis=1) - 4 * field
    ) / (dx * dx)

def gradient(field):
    field_x = (np.roll(field, -1, axis=1) - np.roll(field, 1, axis=1)) / (2 * dx)
    field_y = (np.roll(field, -1, axis=0) - np.roll(field, 1, axis=0)) / (2 * dy)
    return field_x, field_y

def poisson_solve(field):
    for _ in range(50):
        field = (np.roll(field, 1, axis=0) + np.roll(field, -1, axis=0) +
                 np.roll(field, 1, axis=1) + np.roll(field, -1, axis=1)) / 4
    return field

def compute_kinetic_energy(u, v):
    return 0.5 * np.sum(u**2 + v**2)

# Simulation loop
frames = []
time = 0
kinetic_energies = []

while time < t_max:
    # Compute gradients
    rho_x, rho_y = gradient(rho)

    # Update vorticity
    omega += dt * (nu * laplacian(omega) + g * rho_x / rho_heavy)

    # Solve for streamfunction
    psi = poisson_solve(-omega)

    # Compute velocity from streamfunction
    u = gradient(psi)[1]  # u = d(psi)/dy
    v = -gradient(psi)[0]  # v = -d(psi)/dx

    # Maintain initial boundary values
    rho[0, :] = initial_top
    rho[-1, :] = initial_bottom
    rho[:, 0] = initial_left
    rho[:, -1] = initial_right

    # Compute and store kinetic energy
    kinetic_energies.append(compute_kinetic_energy(u, v))

    # Advect density field
    rho -= dt * (u * gradient(rho)[0] + v * gradient(rho)[1])

    # Clamp density values
    rho = np.clip(rho, rho_light, rho_heavy)

    # Save frames less frequently
    if int(time / dt) % 1000 == 0:  # Render every 1000 timesteps
        frames.append(rho.copy())

    time += dt



In [34]:
# Animation
fig, ax = plt.subplots(figsize=(int(nx/25), int(ny/25)))
cax = ax.contourf(X, Y, frames[0], levels=50, cmap="coolwarm")
cb = fig.colorbar(cax, ax=ax)
cb.set_label("Density")

def update(frame):
    ax.clear()
    cax = ax.contourf(X, Y, frame, levels=50, cmap="coolwarm")
    ax.set_title("Rayleigh-Taylor Instability")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    return cax

ani = FuncAnimation(fig, update, frames=frames, interval=200)  # Slower playback
plt.close()
HTML(ani.to_html5_video())




## Expanded Math Explanation (By Jack)

### Governing Equations

1. **Navier-Stokes Equation**:
   The Navier-Stokes equation governs the motion of the fluid, accounting for the effects of pressure, viscosity, and external forces like gravity:
   $
   \frac{\partial \mathbf{v}}{\partial t} + (\mathbf{v} \cdot \nabla)\mathbf{v} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{v} + \mathbf{g}
   $
   In this simulation, the vorticity $(\omega)$ is computed and evolved as:
   $
   \frac{\partial \omega}{\partial t} + \mathbf{v} \cdot \nabla \omega = \nu \nabla^2 \omega + \frac{g}{\rho} \frac{\partial \rho}{\partial x}
   $
   - **Vorticity Transport**: This equation describes the rotation of fluid elements and is updated iteratively.
   - **Coupling to Density**: The gradient of the density field $(\nabla \rho)$ introduces buoyancy effects into the vorticity evolution.

2. **Streamfunction-Velocity Relationship**:
   To satisfy the incompressibility condition $(\nabla \cdot \mathbf{v} = 0)$, the streamfunction $(\psi)$ is used:
   $
   \nabla^2 \psi = -\omega
   $
   The velocity components are then derived from the streamfunction:
   $
   u = \frac{\partial \psi}{\partial y}, \quad v = -\frac{\partial \psi}{\partial x}
   $
   This ensures that the velocity field remains divergence-free.

3. **Advection of Density**:
   The density field is advected by the velocity field:
   $
   \frac{\partial \rho}{\partial t} + \mathbf{v} \cdot \nabla \rho = 0
   $
   This equation ensures that the density evolves with the flow. It is solved using an explicit time-stepping scheme:
   $
   \rho^{n+1} = \rho^n - \Delta t \cdot (\mathbf{v} \cdot \nabla \rho^n)
   $

4. **Kinetic Energy**:
   The total kinetic energy of the system is computed as:
   $
   KE = \frac{1}{2} \sum (u^2 + v^2)
   $
   This provides insights into the energy dynamics of the system.

### Numerical Implementation
- **Grid Discretization**:
  - The simulation domain is divided into a grid with uniform spacing $dx$ and $dy$.
  - Each grid cell represents the state of the fluid (density, velocity, etc.) at that location.

- **Laplacian and Gradients**:
  - The Laplacian operator $(\nabla^2)$ is approximated using finite differences:
    
   $ \nabla^2 f \approx \frac{f_{i+1,j} + f_{i-1,j} + f_{i,j+1} + f_{i,j-1} - 4f_{i,j}}{dx^2}$
    
  - Gradients $(\nabla f)$ are computed similarly for spatial derivatives.

- **Time-Stepping**:
  - An explicit Euler method is used to update all fields. For example:
    $
    \omega^{n+1} = \omega^n + \Delta t \left( \nu \nabla^2 \omega^n + g \frac{\partial \rho^n}{\partial x} \right)
    $




